In [1]:
import os; os.environ["DISPLAY"] = "localhost:10.0"; 
os.environ["__GLX_VENDOR_LIBRARY_NAME"] = "mesa"; # export __GLX_VENDOR_LIBRARY_NAME=mesa   See: https://superuser.com/questions/106056/force-software-based-opengl-rendering-on-ubuntu

import open3d as o3d
import numpy as np
import cv2

from trimesh_render import lookAt

import pytransform3d.transformations as pt

class PointCloudViewer:
    def __init__(self):
        self.origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
        self.eef = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
        self.eef_T_inv = np.eye(4)
        self.pcd = o3d.geometry.PointCloud()

        # Initialize the pointcloud viewer
        self.vis = o3d.visualization.Visualizer()
        self.vis.create_window(window_name="Point Cloud")

        self.vis.add_geometry(self.origin)
        self.vis.add_geometry(self.eef)
        self.vis.add_geometry(self.pcd)

        self.vis.get_render_option().point_size = 1
        # self.vis.get_render_option().background_color = np.asarray([0, 0, 0])

        view_control = self.vis.get_view_control()
        view_control.set_constant_z_far(1000)

        # Retrieve the camera parameters
        camera_params = view_control.convert_to_pinhole_camera_parameters()
        # Set the extrinsic parameters, yz_flip is for Open3D camera configuration
        camera_pose = lookAt(eye=np.array([0., 0., -1.]), target=np.array([0. ,0., 0.]), up=np.array([0.0, -1.0, 0.0]), yz_flip=True)
        camera_params.extrinsic = np.linalg.inv(camera_pose)
        # Set the camera parameters
        view_control.convert_from_pinhole_camera_parameters(camera_params)
        
        frame = self.vis.capture_screen_float_buffer(do_render=True)
        frame = np.asarray(frame)
        self.video_writer = cv2.VideoWriter("demo.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame.shape[1], frame.shape[0]))
    
    def update(self, image_array, depth_array, intrinsics, Teef2cam):
        im_rgb = o3d.geometry.Image(cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB))
        im_d = o3d.geometry.Image(depth_array)

        im_rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
            im_rgb,
            im_d,
            depth_scale=1000.0,
            convert_rgb_to_intensity=False
        )

        height, width, _ = image_array.shape

        new_pcd = o3d.geometry.PointCloud.create_from_rgbd_image(im_rgbd, o3d.camera.PinholeCameraIntrinsic(width=width, height=height, intrinsic_matrix=intrinsics))

        # new_pcd.transform(CAMERA_TO_WORLD)

        self.pcd.points = new_pcd.points
        self.pcd.colors = new_pcd.colors
        self.vis.update_geometry(self.pcd)

        self.eef.transform(self.eef_T_inv)
        self.eef.transform(Teef2cam)
        self.eef_T_inv = pt.invert_transform(Teef2cam)
        self.vis.update_geometry(self.eef)

        # Update the visualizer
        self.vis.poll_events()
        self.vis.update_renderer()
        
        # Capture the current frame
        frame = self.vis.capture_screen_float_buffer(do_render=True)
        frame = np.asarray(frame)
        
        # Convert to uint8 and BGR format for OpenCV
        frame = (frame * 255).astype(np.uint8)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        # Write frame to video
        self.video_writer.write(frame)
    
    def close(self):
        self.video_writer.release()
        self.vis.destroy_window()

viewer = PointCloudViewer()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [2]:
import json
import cv2

save_dir = "output/IPRL+7790ec0a+2023-06-27-21h-02m-21s"

with open(save_dir + "/" + "intrinsics.json", "r") as f:
    intrinsics = json.load(f)

image_array = cv2.imread(save_dir + "/" + "image.png")
depth_array = cv2.imread(save_dir + "/" + "depth.png", cv2.IMREAD_UNCHANGED)

with open(save_dir + "/" + "XYZRPYeef2cams.json", "r") as f:
    XYZRPYeef2cams = np.array(json.load(f))

In [3]:
from utils import xyz_from_uvd, uvd_from_xyz, T_from_xyzrpy, xyzrpy_from_T

# Example usage
for idx, XYZRPYeef2cam in enumerate(XYZRPYeef2cams):
    Teef2cam = T_from_xyzrpy(XYZRPYeef2cam)
    viewer.update(image_array, depth_array, intrinsics, Teef2cam)
    
viewer.close()